In [1]:
import pandas as pd
import numpy as np
import pymysql as py

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#pd.set_option('display.max_rows', 20)

# Data processing

In [6]:
df1=pd.read_csv(r"D:\Python dataset\2023\CSV\All Queires - Jan'23 Final.csv",encoding='cp1252')
df2=pd.read_csv(r"D:\Python dataset\2023\CSV\All Queries - Feb'23 Final.csv",encoding='cp1252')
df3=pd.read_csv(r"D:\Python dataset\2023\CSV\All Queries - Mar'23 Final.csv",encoding='cp1252')
df4=pd.read_csv(r"D:\Python dataset\2023\CSV\All Queries - Apr'23 Final.csv",encoding='cp1252')


C:\Users\ks23007\AppData\Local\Temp\ipykernel_13780\324030792.py:1: DtypeWarning: Columns (4,13,14,17,18,22,23,24,25,26,27,29,30,31,32,33,34,35,36,49,50,51,53,54,55,56,57,58,60,61,65,72,73,74,75,77) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv(r"D:\Python dataset\2023\CSV\All Queires - Jan'23 Final.csv",encoding='cp1252')
C:\Users\ks23007\AppData\Local\Temp\ipykernel_13780\324030792.py:2: DtypeWarning: Columns (4,13,14,17,18,22,23,24,25,26,27,29,30,31,32,33,34,35,36,53,54,55,56,57,58,60,61,65,66,72,73,74,75,77) have mixed types. Specify dtype option on import or set low_memory=False.
  df2=pd.read_csv(r"D:\Python dataset\2023\CSV\All Queries - Feb'23 Final.csv",encoding='cp1252')
C:\Users\ks23007\AppData\Local\Temp\ipykernel_13780\324030792.py:3: DtypeWarning: Columns (0,4,13,14,17,18,22,23,24,25,26,27,29,30,31,32,33,34,35,36,49,50,51,53,54,55,56,57,58,60,61,65,72,73,74,75,77) have mixed types. Specify dtype option on import or set low_mem

In [7]:
df_queries = pd.concat([df1,df2,df3,df4])
df_queries.shape
df_queries = df_queries.drop_duplicates()
df_queries.shape

(2645920, 82)

(2645830, 82)

In [8]:
df_queries = df_queries[['User Id','Query Id', 'Created Date','Client Code', 'SRNature', 'Severity','Escalation Level'
                    ,'Product', 'CallType Name', 'SubCallType Name', 'Source','TAT', 'Last Updated DateTime', 'Month','Status', 'Source2']]

KeyError: "['Month', 'Source2'] not in index"

In [ ]:
df_queries.shape

In [ ]:
df_queries.head()

In [ ]:
df = df_queries

In [ ]:
df['CallType Name'] = df['CallType Name'].str.strip()
df['SubCallType Name'] = df['SubCallType Name'].str.strip()
df['Source2'] = df['Source2'].str.strip()
df['Product'] = df['Product'].str.strip()

## Removing queries with 
### calltype = ['Auto Disposition']
### subcalltype = ['Auto Disposition', 'Auto Dispose', 'No response from client']
### source2 = ['Self service', 'Branch', 'Escalation' ]
### product = ['Dummy_KSEC']

In [ ]:
calltype = ['Auto Disposition']
subcalltype = ['Auto Disposition', 'Auto Dispose', 'No response from client']
source2 = ['Self service', 'Branch', 'Escalation' ]
product = ['Dummy_KSEC']

In [ ]:
df = df[~df['CallType Name'].isin(calltype)]
df = df[~df['SubCallType Name'].isin(subcalltype)]
df = df[~df['Source2'].isin(source2)]
df = df[~df['Product'].isin(product)]  

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.rename(columns = {'Client Code':'clientcode'}, inplace = True)

In [ ]:
# Columns
list(df.columns.values)

# Source analysis

In [ ]:
pd.pivot_table(df, index="Month", columns="Source2",values="Query Id", aggfunc=np.count_nonzero)

In [ ]:
# Value counts
pd.DataFrame(df['Source2'].value_counts())

In [ ]:
#Distinct counts
pd.DataFrame(df.groupby('Source2').clientcode.nunique().sort_values(ascending=False))

# Mapping source2 to Phone/Email/Livechat

In [ ]:
df.head()

In [ ]:
df['channel'] = np.where(df['Source2'].isin(['Inbound', 'Outbound']), 'Phone',
                   np.where(df['Source2'].isin(['Email','PCG']), 'Email',
                   np.where(df['Source2'].isin(['Chat']), 'Livechat', 'Others')))

# Overall counts

In [ ]:
pd.DataFrame(df.groupby(['channel']).clientcode.nunique().sort_values(ascending=False))

In [ ]:
pd.DataFrame(df['channel'].value_counts())

# Channel analysis

In [ ]:
df2=df[df['channel'].isin(['Phone','Email','Livechat'])]

In [ ]:
df2.shape

In [ ]:
df2.head()

In [ ]:
#Distinct counts
pd.DataFrame(df2.groupby(['Month','channel']).clientcode.nunique().sort_values(ascending=False))

In [ ]:
# Value counts
pd.DataFrame(df2.groupby(['Month','channel']).value_counts())

# MOM Analysis

In [ ]:
# query counts
pd.DataFrame(df2.groupby(['Month', 'channel']).clientcode.count().sort_values(ascending=False))

In [ ]:
# client counts
pd.DataFrame(df2.groupby(['Month', 'channel']).clientcode.nunique().sort_values(ascending=False))


In [ ]:
#unique counts and count1
df2.groupby([ 'channel','Month']).agg({'clientcode': ['nunique', 'count']})

# MOM top queries

In [ ]:
# Value counts
pd.DataFrame(df2['Product'].value_counts())

In [ ]:
df2['query_count']=1
df2.head()

In [ ]:
pivot_merge = pd.pivot_table(df2,index=['Month', 'clientcode', 'Product', 'channel'],values=['query_count'],aggfunc=np.sum)
pivot_merge.reset_index(inplace=True)
pivot_merge.head()
pivot_merge.shape

# Repeat Customers

In [ ]:
df_repeat = df2.groupby([ 'Month','clientcode']).agg({'channel': ['nunique']})
df_repeat=df_repeat.reset_index()
df_repeat.columns = df_repeat.columns.droplevel()
df_repeat.columns = ['month', 'clientcode', 'no_of_sources']

In [ ]:
df_repeat.columns

In [ ]:
df_repeat.head()

# Omni channel Analysis

In [ ]:
df_repeat[df_repeat['no_of_sources']>=3].groupby([ 'month']).agg({'clientcode': ['count']}).transpose()

In [ ]:
df_repeat[df_repeat['no_of_sources']==2].groupby([ 'month']).agg({'clientcode': ['count']}).transpose()

In [ ]:
df_repeat[df_repeat['no_of_sources']==1].groupby([ 'month']).agg({'clientcode': ['count']}).transpose()

### Repeat clients queries

In [ ]:
df3=df2.merge(df_repeat, on='clientcode', how='left')

In [ ]:
df3.head()

In [ ]:
df3.columns

In [ ]:
pd.DataFrame(df3[df3['no_of_sources']==3].groupby('Product').clientcode.nunique().sort_values(ascending=False))

In [ ]:
df2['query']  = df2['Product']+'/'+df2['CallType Name']+'/'+df2['SubCallType Name']

In [ ]:
df2.shape

In [ ]:
### Avg query per client

In [ ]:
pd.DataFrame(df2.groupby(['clientcode']).query_count.sum().sort_values(ascending=False)).head()

In [ ]:
df_clean_queries=df2[~df2['clientcode'].isin(['123NW', '123DP'])]

In [ ]:
df_clean_queries.dtypes

In [ ]:
df_clean_queries['Created Date'] = pd.to_datetime(df_clean_queries['Created Date'])

In [ ]:
df_clean_queries.head()

In [ ]:
# Total no of queries no of source wise MOM
for i in ['Jun-22', 'Jul-22','Aug-22']:
    print(i)
    pd.DataFrame(df_clean_queries.loc[df_clean_queries.clientcode.isin(df_repeat[(df_repeat['no_of_sources']==2) & (df_repeat['month']==i)]['clientcode'])]
            ).clientcode.nunique()

In [ ]:
# No of queries no_of_sources wise MOM
for i in ['Jun-22', 'Jul-22','Aug-22']:
    print(i)
    pd.DataFrame(df_clean_queries.loc[df_clean_queries.clientcode.isin(df_repeat[(df_repeat['no_of_sources']==1) & (df_repeat['month']==i)]['clientcode'])]
            ).groupby(['Month']).query_count.sum().sort_values(ascending=False)

In [ ]:
#query by clients using >=2 channels 

In [ ]:
# No of queries no_of_sources wise MOM
pd.DataFrame(df_clean_queries.loc[df_clean_queries.clientcode.isin(df_repeat[(df_repeat['no_of_sources']==2) & (df_repeat['month']=='Aug-22')]['clientcode'])]
            ).groupby(['Month','query']).query_count.sum().sort_values(ascending=False).head(10)

In [ ]:
pd.DataFrame(df_clean_queries.loc[df_clean_queries.clientcode.isin(df_repeat[(df_repeat['no_of_sources']==2) & (df_repeat['month']=='Jun-22')]['clientcode'])]
            ).groupby(['Month','query']).clientcode.nunique().sort_values(ascending=False).head(10)

In [ ]:
pd.DataFrame(df_clean_queries.loc[df_clean_queries.clientcode.isin(df_repeat[(df_repeat['no_of_sources']==2) & (df_repeat['month']=='Jul-22')]['clientcode'])]
            ).groupby(['Month','query']).clientcode.nunique().sort_values(ascending=False).head(10).to_clipboard()

In [ ]:
multiple_query= df_clean_queries.groupby(['Month','clientcode', 'query']).agg({'query_count': 'sum', 'channel': 'nunique', 'Created Date':['min','max'] })

In [ ]:
multiple_query.reset_index(inplace=True)

In [ ]:
multiple_query.columns = multiple_query.columns.droplevel()

In [ ]:
multiple_query.columns = [    'Month','clientcode', 'query', 'query_count', 'channel', 'min_date', 'max_date']

In [ ]:
multiple_query['avg_time'] = (multiple_query['max_date'] - multiple_query['min_date'])/multiple_query['query_count']

In [ ]:
multiple_query['avg_time']= round(multiple_query['avg_time'].dt.total_seconds()/(3600.0),0)

In [ ]:
multiple_query[multiple_query['query_count']>1].head()

In [ ]:
#Total queries
multiple_query.groupby(['Month']).query_count.sum()

# Repeating queries

In [ ]:
multiple_query[multiple_query['query_count']>=2].groupby(['Month']).query_count.sum()

In [ ]:
for i in ['Jun-22', 'Jul-22','Aug-22']:
    multiple_query[(multiple_query['Month']==i) & (multiple_query['query_count']>=2) ].groupby(['Month', 'channel']
    ).query_count.sum().sort_values(ascending=False).head()

In [ ]:
df_1 = pd.DataFrame(multiple_query[(multiple_query['Month']=='Jun-22') & (multiple_query['query_count']>=2) ].groupby(['Month', 'channel','query']
    ).agg({'query_count': 'sum', 'clientcode': 'nunique', 'avg_time': 'mean'}))
df_1.reset_index(inplace=True)

In [ ]:
df_2 = pd.DataFrame(multiple_query[(multiple_query['Month']=='Jul-22') & (multiple_query['query_count']>=2) ].groupby(['Month', 'channel','query']
    ).agg({'query_count': 'sum', 'clientcode': 'nunique','avg_time': 'mean'}))
df_2.reset_index(inplace=True)

In [ ]:
df_3 = pd.DataFrame(multiple_query[(multiple_query['Month']=='Aug-22') & (multiple_query['query_count']>=2) ].groupby(['Month', 'channel','query']
    ).agg({'query_count': 'sum', 'clientcode': 'nunique','avg_time': 'mean'}))
df_3.reset_index(inplace=True)

In [ ]:
df_merge = pd.concat([df_1, df_2, df_3], ignore_index=True)

In [ ]:
df_merge=df_merge.sort_values(by = 'query_count', ascending=False)

In [ ]:
df_merge.head()

In [ ]:
for i in ['Jun-22', 'Jul-22','Aug-22']:
    pd.DataFrame(multiple_query[(multiple_query['Month']==i) & (multiple_query['query_count']>=2) ].groupby(['Month', 'channel']
    ).agg({'query_count': 'sum', 'clientcode': 'nunique', 'avg_time': 'mean'})).sort_values(by = 'query_count' , ascending=False).head(5)

In [ ]:
for i in ['Jun-22', 'Jul-22','Aug-22']:
    for y in [1,2,3]:
        pd.DataFrame(multiple_query[(multiple_query['Month']==i) & (multiple_query['query_count']>=2) & (multiple_query['channel']==y) ].groupby(['Month', 'channel','query']
    ).agg({'query_count': 'sum', 'clientcode': 'nunique', 'avg_time': 'mean'})).sort_values(by = 'query_count' , ascending=False).head(5).reset_index()

In [ ]:
pd.DataFrame(multiple_query[(multiple_query['Month']=='Aug-22') & (multiple_query['query_count']>=2) 
                            & (multiple_query['channel']==3) ])

In [ ]:
multiple_query[(multiple_query['Month']=='Aug-22') & (multiple_query['query_count'] >2)
                 & (multiple_query['query']=='Process & Updation [New]/Process and Updation/Activations in FNO Segment')]

In [ ]:
df_clean_queries[(df_clean_queries['clientcode']=='8TC26')
                 & (df_clean_queries['query']=='Process & Updation [New]/Process and Updation/Activations in FNO Segment')].head()